In [16]:
import json
import os
from utils import create_driver
import time, random
from tqdm import tqdm

from google_scholar import scrape_publications, search_from_google_scholar_profile

In [12]:
dir = './raw_data'
files = [os.path.join(dir, f) for f in os.listdir(dir) if os.path.isfile(os.path.join(dir, f)) and 'dr_ntu' in f]
names = []

for file in files:
    with open(file, 'r') as f:
        profile = json.load(f)
    if profile['google_scholar'] is None:
        names.append(profile['full_name'])
len(names)

54

In [14]:
# Search for profile using google scholar profile search

profile_search_url = 'https://scholar.google.com/citations?view_op=search_authors&hl=en'

driver = create_driver(debug=True)

driver.get(profile_search_url)
time.sleep(random.uniform(2,4))

results = {}
for name in tqdm(names):
    results[name] = search_from_google_scholar_profile(name, driver)
    time.sleep(random.uniform(1,3))
    if random.uniform(0,1)>0.75:
        time.sleep(random.uniform(5,7))

    if random.uniform(0,1)>0.8:
        driver.quit()
        driver = create_driver(debug=False)
        driver.get(profile_search_url)
        time.sleep(random.uniform(2,4))

driver.quit()

In [18]:
for name in results.keys():
    values = results[name]
    pubs = []
    with open(f'./raw_data/dr_ntu_{name.lower().replace(" ", "_")}.json', 'r') as f:
        dr_ntu = json.load(f)

    for type in ['articles', 'books','book_chapters', 'conferences']:
        pubs += dr_ntu[type]
    values = {'possible_google_scholar_url': values, 'dr_ntu_publications': pubs}
    
    results[name] = values

In [19]:
possible_google_scholar = {}
for k, v in results.items():
    links = v['possible_google_scholar_url']
    possible_google_scholar.update({url:{'full_name': k} for url in links})

In [21]:
driver = create_driver(debug=True)
for url in tqdm(possible_google_scholar.keys()):
    driver.get(url)
    time.sleep(random.uniform(2,5))

    g_publication = scrape_publications(driver)

    possible_google_scholar[url]['google_publications'] = g_publication

    if random.uniform(0,1)>0.8:
        time.sleep(random.uniform(5,10))
    
    if random.uniform(0,1)>0.9:
        driver.quit()
        driver = create_driver(debug=False)

driver.quit()

100%|██████████| 43/43 [13:07<00:00, 18.30s/it]


In [22]:
for g_scholar_link, values in possible_google_scholar.items():
    name = values['full_name']
    g_pubs = values['google_publications']

    dr_ntu_pubs = results[name]['dr_ntu_publications']
    if len(dr_ntu_pubs)==0:
        continue
    cnt = 0
    for title in dr_ntu_pubs:
        for g_pub in g_pubs:
            if g_pub['title'].lower() in title.lower():
                cnt +=1
                break
    if cnt/len(dr_ntu_pubs)>0:
        if 'google_scholar' not in results[name]:
            results[name]['google_scholar'] = {'link':g_scholar_link, 'score': cnt/len(dr_ntu_pubs)}
        elif results[name]['google_scholar']['score']<cnt/len(dr_ntu_pubs):
            results[name]['google_scholar'] = {'link':g_scholar_link, 'score': cnt/len(dr_ntu_pubs)}
        

In [23]:
google_scholar = {name:v['google_scholar']['link'] for name, v in results.items() if 'google_scholar' in v}

In [24]:
len(google_scholar)

34

In [25]:
cnt = 0

for k,i in results.items():
    if 'google_scholar' not in i and len(i['possible_google_scholar_url'])>0:
        print(k)
        print(i['possible_google_scholar_url'])
        print(i['dr_ntu_publications'])
        cnt+=1
cnt

Lin Guosheng
['https://scholar.google.com/citations?hl=en&user=ZudEhvcAAAAJ']
[]
Liu Ziwei
['https://scholar.google.com/citations?hl=en&user=lc45xlcAAAAJ']
[]
Qian Kemao
['https://scholar.google.com/citations?hl=en&user=JCehBDoAAAAJ']
[]
Mohamed M. Sabry
['https://scholar.google.com/citations?hl=en&user=eCo7XWkAAAAJ']
[]
Vidya Sudarshan
['https://scholar.google.com/citations?hl=en&user=jhD-lGAAAAAJ']
['Yu Na Wu, Vidya K Sudarshan, Shi Chao Zhu, Sung Joon Kim, Yin Hua Zhang. Functional interactions between complex I and nNOS in regulating cardiac mitochondrial activity in sham and hypertensive rat hearts. Pflugers Archiv – European Journal of Physiology, 2020; 472: 1743-1755.', 'Vidya K Sudarshan, U Rajendra Acharya, Oh Shu Lih, Muhammad Adam, Tan Jen Hong, Chua Kuang Chua, Chua Kok Poo, Tan Ru San. Automated diagnosis of congestive heart failure using dual tree complex wavelet transform and statistical features extracted from 2 seconds of ECG signals. Computers in Biology and Medicine,

5

In [27]:
for name in google_scholar:
    with open(f'./raw_data/dr_ntu_{name.lower().replace(" ", "_")}.json', 'r') as f:
        dr_ntu = json.load(f)
    dr_ntu['google_scholar'] = google_scholar[name]

    with open(f'./raw_data/dr_ntu_{name.lower().replace(" ", "_")}.json', 'w') as f:
        json.dump(dr_ntu, f)